In [1]:
import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import re
from bs4 import BeautifulSoup

In [2]:
driver = webdriver.Chrome()

In [3]:
main_page_url = "https://betsapi.com/te/22202/Galatasaray"
driver.get(main_page_url)
driver.get(main_page_url)

In [4]:
import re
wait = WebDriverWait(driver, 10)

matches_list = []
page_numbers = []

element_present = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'table')))
soup = BeautifulSoup(driver.page_source, 'html.parser')
page_buttons = soup.find_all("a", class_="page-link")
# Extract the number of pages from each url
if len(page_buttons) > 0:
    page_numbers = [int(re.sub(r'\D', '', button.get_text())) for button in page_buttons if button.get_text().isdigit()]
    max_page_number = max(page_numbers) if page_numbers else 1
else:
    max_page_number = 1
for page_number in range(1, max_page_number + 1):
     # Cycle through each page of the url and get each page's content
    if page_number != 1:
        url = f"{main_page_url}/p.{page_number}"  # Construct the URL
        driver.get(url)
        element_present = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'table')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')   
    # Extract the relevant data from the page like item name, price, etc.
    rows = soup.findAll('tr')
    if len(rows) > 0:
        for row in rows:
            league_name = row.find("td", class_="league_n").text.strip()
            date = row.find("td", class_="dt_n").text.strip()
            league_match_no = row.find_all("td")[2].text.strip()
            f_place = row.find_all("sup")[0].text.strip() if row.find_all("sup") else ""
            team_names = row.find_all("td")[3].text.strip()
            #f_team_name = team_info.split("v")[0] 
            #s_team_name = team_info.split("v")[1] 
            s_place = row.find_all("sup")[1].text.strip() if row.find_all("sup") else ""
            badge = [td.text.strip() for td in row.find_all("td") if td.get("class") and "badge_" in " ".join(td.get("class"))]
            result = row.find_all("td")[5].text.strip()
            matches_list.append((league_name, date, league_match_no, f_place, team_names, s_place, badge, result))
    
# Close the WebDriver
driver.quit()

col_titles = ['LeagueName', 'Date', 'LeagueMatchNo', 'F_Place', 'Team_Names', 'S_Place', 'Badge', 'Result']
df = pd.DataFrame(matches_list, columns=col_titles)

df

,LeagueName,Date,LeagueMatchNo,F_Place,Team_Names,S_Place,Badge,Result
0,Turkey Super Cup,04/07 11:30,1,,Galatasaray\n v\n ...,,[W],DBFA
1,Turkey Super Cup,04/07 00:16,-,,Galatasaray\n v\n ...,,[W],1-0
2,Turkey Super Lig,04/02 10:30,31,[1],[1] Galatasaray\n ...,[15],[W],1-0
3,Turkey Super Lig,03/17 06:00,30,[5],[5] Kasimpasa\n v\n ...,[1],[W],3-4
4,Turkey Super Lig,03/08 09:00,29,[1],[1] Galatasaray\n ...,[7],[W],6-2
...,...,...,...,...,...,...,...,...
542,World Club Friendlies,2014-07-27,-,,Budapest Honved\n v\n ...,,[W],1-2
543,World Club Friendlies,2014-07-23,-,,Rapid Vienna\n v\n ...,,[L],3-1
544,Turkey Cup,2014-05-07,1,,Eskisehirspor\n v\n ...,,[W],0-1
545,Turkey Cup,2014-04-16,2,,Bursaspor\n v\n ...,,[W],2-5


In [6]:
col_titles = ['LeagueName', 'Date', 'LeagueMatchNo', 'F_Place', 'Team_Names', 'S_Place', 'Badge', 'Result']
df = pd.DataFrame(matches_list, columns=col_titles)

df

,LeagueName,Date,LeagueMatchNo,F_Place,Team_Names,S_Place,Badge,Result
0,Turkey Super Cup,04/07 11:30,1,,Galatasaray\n v\n ...,,[W],DBFA
1,Turkey Super Cup,04/07 00:16,-,,Galatasaray\n v\n ...,,[W],1-0
2,Turkey Super Lig,04/02 10:30,31,[1],[1] Galatasaray\n ...,[15],[W],1-0
3,Turkey Super Lig,03/17 06:00,30,[5],[5] Kasimpasa\n v\n ...,[1],[W],3-4
4,Turkey Super Lig,03/08 09:00,29,[1],[1] Galatasaray\n ...,[7],[W],6-2
...,...,...,...,...,...,...,...,...
542,World Club Friendlies,2014-07-27,-,,Budapest Honved\n v\n ...,,[W],1-2
543,World Club Friendlies,2014-07-23,-,,Rapid Vienna\n v\n ...,,[L],3-1
544,Turkey Cup,2014-05-07,1,,Eskisehirspor\n v\n ...,,[W],0-1
545,Turkey Cup,2014-04-16,2,,Bursaspor\n v\n ...,,[W],2-5


In [7]:
df.to_csv("C:/Users/sevgi/OneDrive/Masaüstü/csis/winter 2024/csis4260/Project/gs_matches.csv", index=False, encoding='utf-8')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547 entries, 0 to 546
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   LeagueName     547 non-null    object
 1   Date           547 non-null    object
 2   LeagueMatchNo  547 non-null    object
 3   F_Place        547 non-null    object
 4   Team_Names     547 non-null    object
 5   S_Place        547 non-null    object
 6   Badge          547 non-null    object
 7   Result         547 non-null    object
dtypes: object(8)
memory usage: 34.3+ KB
